In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten, Add, GlobalAveragePooling2D,Input, BatchNormalization, Activation, concatenate,AveragePooling2D,Softmax
from keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.regularizers import L1L2,L2
import os
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
import keras_cv
from tensorflow.keras.applications import InceptionV3,InceptionResNetV2

In [ ]:
(train_x,train_y),(test_x,test_y) = tf.keras.datasets.cifar10.load_data()
print(type(train_y))

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
train_x = train_x.astype('float32') / 255
test_x = test_x.astype('float32') / 255
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)
train_x ,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=.1)

In [ ]:
data_augmentation = Sequential(
    [
        layers.Normalization(),
    ]
)
data_augmentation.layers[0].adapt(train_x)

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_data = datagen.flow(train_x,train_y,batch_size=128)

In [ ]:
def inception_block(x,filter_1,filter_3_reduce,filter_3,filter_5_reduce,filter_5,filter_pool):
    conv_1 = Conv2D(filter_1,kernel_size=1,padding='same')(x)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = Activation('relu')(conv_1)
    conv_3 = Conv2D(filter_3_reduce,kernel_size=1,padding='same')(x)
    conv_3 = BatchNormalization()(conv_3)
    conv_3 = Activation('relu')(conv_3)
    conv_3 =  Conv2D(filter_3,kernel_size=3,padding='same')(conv_3)
    conv_3 = BatchNormalization()(conv_3)
    conv_3 = Activation('relu')(conv_3)
    conv_5 = Conv2D(filter_5_reduce,kernel_size=1,padding='same')(x)
    conv_5 = BatchNormalization()(conv_5)
    conv_5 = Activation('relu')(conv_5)
    conv_5 = Conv2D(filter_5,kernel_size=5,padding='same',activation='relu')(conv_5)
    conv_5 = BatchNormalization()(conv_5)
    conv_5 = Activation('relu')(conv_5)
    pool = MaxPooling2D(3,padding='same',strides=1)(x)
    pool = conv_1 = Conv2D(filter_pool,kernel_size=1,padding='same')(pool)
    pool = BatchNormalization()(pool)
    pool = Activation('relu')(pool)
    output = concatenate([conv_1,conv_3,conv_5,pool],axis=3)
    return output
def get_output(x,name):
    x = AveragePooling2D((3,3),padding='same')(x)
    x = Conv2D(128,kernel_size=1,padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Flatten()(x)
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(.4)(x)
    x = Dense(10, activation='softmax',name=name)(x)
    return x
    
def inception_net(in_shape):
    inputlayer = Input(in_shape)
    x = data_augmentation(inputlayer)
    x = Conv2D(64,kernel_size=7,padding='same',strides=2)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(3,padding='same',strides=2)(x)
    x = Conv2D(64,kernel_size=1,padding='same',strides=1)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(192,kernel_size=3,padding='same',strides=1)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(3,padding='same',strides=2)(x)
    x = inception_block(x,64,96,128,16,32,32)
    x = inception_block(x,128,128,196,32,96,64)
    x = MaxPooling2D(3,padding='same',strides=2)(x)
    x = inception_block(x,192,96,208,16,48,64)
    output_1 = get_output(x,"output_1")
    x = inception_block(x,160,112,224,24,64,64)
    x = inception_block(x,128,128,256,24,64,64)
    x = inception_block(x,112,144,288,32,64,64)
    output_2 = get_output(x,"output_2")
    x = inception_block(x,256,160,320,32,128,128)
    x = MaxPooling2D(3,padding='same',strides=2)(x)
    x = inception_block(x,256,160,320,32,128,128)
    x = inception_block(x,384,192,384,48,128,128)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    x = Dense(500)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    output_3 = Dense(10, activation='softmax', name='output_3')(x)
    return inputlayer,[output_1,output_2,output_3]
    

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(.01,decay_steps=1000,decay_rate=0.96)
print(type(train_x))
inputlayer,output = inception_net((32,32,3)) 
inception = Model(inputlayer,output)

inception.compile(optimizer=Adam(learning_rate=lr_schedule,weight_decay=1e-4),loss='categorical_crossentropy',metrics=['categorical_accuracy'])
m1= inception.fit(train_data,epochs=200,validation_data=(val_x,val_y))

In [ ]:
plt.figure(figsize=(20,14))
sb.set_style("whitegrid")

plt.plot(m1.history['output_1_categorical_accuracy'])
plt.plot(m1.history['output_2_categorical_accuracy'])
plt.plot(m1.history['output_3_categorical_accuracy'])
plt.plot(m1.history['val_output_1_categorical_accuracy'])
plt.plot(m1.history['val_output_2_categorical_accuracy'])
plt.plot(m1.history['val_output_3_categorical_accuracy'])
plt.title('Loss comparison between Validation and Train Data set',fontsize=15)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train1','train2','train3', 'val1','val2','val3'], loc='best')
plt.show()

In [ ]:
inception.evaluate(test_x,test_y)
print(train_y.shape)